## Read data from database

In [1]:
import os
# specify which GPU will be used
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
import os

import pymysql
from warnings import filterwarnings

_connection = None

def get_connection(db_config):
    """
    get db connection
    :return:
    """
    global _connection
    if _connection is None:
        _connection = pymysql.connect(host=db_config['host'], user=db_config['username'],
                                      password=db_config['password'],
                                      db=db_config['db'], charset="utf8")
        filterwarnings('ignore', category=pymysql.Warning)

    return _connection


def close():
    """
    close DB connection
    :return:
    """
    global _connection
    if _connection is not None:
        _connection.close()
    _connection = None

In [3]:
db = {
    'host': '172.26.187.242',
    'username': 'malware_r',
    'password': 'GEg22v2O7jbfWhb3',
    'db': 'malware'
}

### Fields

- mw_file_suffix: file name after hash value
- mw_file_prefix: directory
- mw_em_f: features of ember, splitted by ";"

In [4]:
import time

# the base function which can query sql and return dict data
def get_specific_data(table_suffix, sql=None):
    start_time = time.time()
    
    global _connection
    if _connection is None:
        raise Exception("please init db connect first")

    cursor = _connection.cursor()
    cursor.execute("SET NAMES utf8mb4")

    ret = []
        
    cursor.execute(sql)

    field_names = [i[0] for i in cursor.description]

    for row in cursor:
        temp = {}
        for key in range(len(row)):
            temp[field_names[key]] = row[key]
        ret.append(temp)
     
    cursor.close()
    # _connection.close()
    print("--- %s seconds ---" % (time.time() - start_time))
    
    return ret

In [5]:
close()
res1 = []
get_connection(db)
table_suffix = ["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F"]
# Iterate all partitions of databases
for suffix in table_suffix:
    sql = """ 
select
  a.mw_file_hash,
  a.section_name,
  c.mw_file_suffix as mw_file_size,
  c.mw_file_prefix as mw_file_directory,
  c.mw_num_engines,
  a.pointerto_raw_data,
  a.virtual_size,
  d.mw_em_f
from mw_index_2017_section_%s as a
  inner join mw_index_2017_%s c on a.mw_file_hash = c.mw_file_hash
  inner join mw_index_2017_feature_%s d on a.mw_file_hash = d.mw_file_hash
where CNT_CODE = 1 and MEM_EXECUTE = 1 and c.mw_num_engines <> -1 and (c.mw_num_engines > 6 or c.mw_num_engines = 0) and
      c.mw_file_prefix in ('201701', '201702')
group by mw_file_hash
    """ % (suffix, suffix, suffix)
    res1.extend(get_specific_data(suffix, sql))
close()
print(len(res1))

--- 23.17503261566162 seconds ---
--- 24.909729957580566 seconds ---
--- 23.01578664779663 seconds ---
--- 23.8974347114563 seconds ---
--- 20.820040702819824 seconds ---
--- 25.139378309249878 seconds ---
--- 25.73338532447815 seconds ---
--- 24.516175031661987 seconds ---
--- 25.318306922912598 seconds ---
--- 23.870152473449707 seconds ---
--- 22.4960675239563 seconds ---
--- 14.336087942123413 seconds ---
--- 15.16492486000061 seconds ---
--- 12.79147720336914 seconds ---
--- 14.758151531219482 seconds ---
--- 15.459365129470825 seconds ---
268423


In [6]:
close()
res2 = []
get_connection(db)
table_suffix = ["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F"]
# Iterate all partitions of databases
for suffix in table_suffix:
    sql = """ 
select
  a.mw_file_hash,
  a.section_name,
  c.mw_file_suffix as mw_file_size,
  c.mw_file_prefix as mw_file_directory,
  c.mw_num_engines,
  a.pointerto_raw_data,
  a.virtual_size,
  d.mw_em_f
from mw_index_2017_section_%s as a
  inner join mw_index_2017_%s c on a.mw_file_hash = c.mw_file_hash
  inner join mw_index_2017_feature_%s d on a.mw_file_hash = d.mw_file_hash
where CNT_CODE = 1 and MEM_EXECUTE = 1 and c.mw_num_engines <> -1 and (c.mw_num_engines > 6 or c.mw_num_engines = 0) and
      c.mw_file_prefix in ('201703')
group by mw_file_hash
    """ % (suffix, suffix, suffix)
    res2.extend(get_specific_data(suffix, sql))
close()
print(len(res2))

--- 6.196787118911743 seconds ---
--- 8.897745370864868 seconds ---
--- 7.0894858837127686 seconds ---
--- 7.539840936660767 seconds ---
--- 11.23170280456543 seconds ---
--- 15.15299105644226 seconds ---
--- 12.75526237487793 seconds ---
--- 12.518447637557983 seconds ---
--- 14.171578884124756 seconds ---
--- 12.5701744556427 seconds ---
--- 13.190277338027954 seconds ---
--- 12.069642543792725 seconds ---
--- 14.34741735458374 seconds ---
--- 11.93936562538147 seconds ---
--- 12.453815221786499 seconds ---
--- 11.3638014793396 seconds ---
154527


## Check and split data

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import pylab as pl
from sklearn.model_selection import train_test_split

max_length = 300000

train_data = pd.DataFrame(res1)
# train_data = train_data.loc[train_data.virtual_size <= max_length]
# train_data = train_data.reset_index(drop=True)
train_data.mw_num_engines[train_data.mw_num_engines == 0 ] = 0
train_data.mw_num_engines[train_data.mw_num_engines > 6 ] = 1
train_label = train_data.mw_num_engines.ravel()

test_data = pd.DataFrame(res2)
# test_data = test_data.loc[test_data.virtual_size <= max_length]
# test_data = test_data.reset_index(drop=True)
test_data.mw_num_engines[test_data.mw_num_engines == 0 ] = 0
test_data.mw_num_engines[test_data.mw_num_engines > 6 ] = 1
test_label = test_data.mw_num_engines.ravel()

x_train, x_val, y_train, y_val = train_test_split(train_data, train_label, test_size=0.1, random_state=2345)
x_test = test_data
y_test = test_label

x_train = x_train.reset_index(drop=True)
x_val = x_val.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)

/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ip

## EMBER

In [8]:
import pandas as pd
import re
import hashlib
import numpy as np
from sklearn.feature_extraction import FeatureHasher
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss, confusion_matrix

In [9]:
def get_ember_feature(data):
    ember_f = np.zeros((len(data.mw_em_f), 2351), dtype=float)
    for index, item in data.iterrows():
        ember_f[index, :] = item['mw_em_f'].split(';')
    return ember_f

In [10]:
def get_model(x_train, y_train, x_val, y_val):
    params = {'application': 'binary'}
    lgbm_dataset = lgb.Dataset(x_train, y_train.ravel())
    valid_sets = lgb.Dataset(x_val, y_val.ravel())

    model = lgb.train(params, lgbm_dataset, 100000, valid_sets=valid_sets, early_stopping_rounds=10)
    y_pred = model.predict(x_val)
    
    loss = log_loss(y_val, y_pred)
    auc = roc_auc_score(y_val, y_pred)
    acc = accuracy_score(y_val, (y_pred > 0.5).astype(int))
#     model.save_model(file_path + "-%04d-%.5f-%.5f.h5" % (model.best_iteration, loss, acc),
#                      num_iteration=model.best_iteration)
    print("val loss : %.5f" % loss)
    print("auc score : %.5f" % auc)
    print("accuracy score : %.5f" % acc)
        
    return model

In [11]:
x_etrain = get_ember_feature(x_train)
x_eval = get_ember_feature(x_val)
x_etest = get_ember_feature(x_test)

In [12]:
import matplotlib.pyplot as plt
model = get_model(x_etrain, y_train, x_eval, y_val)

[1]	valid_0's binary_logloss: 0.584837
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's binary_logloss: 0.525152
[3]	valid_0's binary_logloss: 0.475066
[4]	valid_0's binary_logloss: 0.434171
[5]	valid_0's binary_logloss: 0.397788
[6]	valid_0's binary_logloss: 0.367165
[7]	valid_0's binary_logloss: 0.33936
[8]	valid_0's binary_logloss: 0.314477
[9]	valid_0's binary_logloss: 0.293052
[10]	valid_0's binary_logloss: 0.274979
[11]	valid_0's binary_logloss: 0.25826
[12]	valid_0's binary_logloss: 0.243444
[13]	valid_0's binary_logloss: 0.230319
[14]	valid_0's binary_logloss: 0.218117
[15]	valid_0's binary_logloss: 0.207164
[16]	valid_0's binary_logloss: 0.196933
[17]	valid_0's binary_logloss: 0.188156
[18]	valid_0's binary_logloss: 0.180111
[19]	valid_0's binary_logloss: 0.173023
[20]	valid_0's binary_logloss: 0.166436
[21]	valid_0's binary_logloss: 0.160319
[22]	valid_0's binary_logloss: 0.154994
[23]	valid_0's binary_logloss: 0.149537
[24]	valid_0's binary_logloss

[199]	valid_0's binary_logloss: 0.0691758
[200]	valid_0's binary_logloss: 0.0690944
[201]	valid_0's binary_logloss: 0.0690698
[202]	valid_0's binary_logloss: 0.06899
[203]	valid_0's binary_logloss: 0.0689309
[204]	valid_0's binary_logloss: 0.0688958
[205]	valid_0's binary_logloss: 0.0688629
[206]	valid_0's binary_logloss: 0.068872
[207]	valid_0's binary_logloss: 0.0688608
[208]	valid_0's binary_logloss: 0.0688034
[209]	valid_0's binary_logloss: 0.0687467
[210]	valid_0's binary_logloss: 0.0687248
[211]	valid_0's binary_logloss: 0.0687062
[212]	valid_0's binary_logloss: 0.0687077
[213]	valid_0's binary_logloss: 0.0686773
[214]	valid_0's binary_logloss: 0.0686425
[215]	valid_0's binary_logloss: 0.0686317
[216]	valid_0's binary_logloss: 0.0686314
[217]	valid_0's binary_logloss: 0.0686491
[218]	valid_0's binary_logloss: 0.0687027
[219]	valid_0's binary_logloss: 0.0686796
[220]	valid_0's binary_logloss: 0.0686566
[221]	valid_0's binary_logloss: 0.0686633
[222]	valid_0's binary_logloss: 0.068

In [12]:
def estimate_model(y_pred, test_y):
    
    loss = log_loss(test_y, y_pred)
    auc = roc_auc_score(test_y, y_pred)
    acc = accuracy_score(test_y, (y_pred > 0.5).astype(int))
    print("loss : %.5f" % loss)
    print("auc score : %.5f" % auc)
    print("accuracy score : %.5f" % acc)

    fp_np_index = np.where(test_y == 0)
    fp_np = y_pred[fp_np_index].shape[0]
    thre_index = int(np.ceil(fp_np - fp_np * 0.001))

    sorted_pred_prob = np.sort(y_pred[fp_np_index], axis=0)
    thre = sorted_pred_prob[thre_index]
    if thre == 1:
        thre = max(sorted_pred_prob[np.where(sorted_pred_prob != 1)])

    y_pred_prob = np.vstack((y_pred.transpose(), (1 - y_pred).transpose())).transpose()
    y_pred_prob[:, 1] = thre
    y_pred_label = np.argmin(y_pred_prob, axis=-1)

    tn, fp, fn, tp = confusion_matrix(test_y, y_pred_label).ravel()
    fp_rate = fp / (fp + tn)
    recall_rate = tp / (tp + fn)

    print("thre: %.10f"%  thre)
    print("fp:  %.10f"%  fp_rate)
    print("recall:  %.10f"%  recall_rate)
    
    return auc, loss, recall_rate

In [14]:
y_p = model.predict(x_etest)
y_pred_e = np.zeros((len(y_p), 1))
for i in range(len(y_p)):
    y_pred_e[i, 0] = y_p[i]

estimate_model(y_pred_e, y_test)

loss : 0.25842
auc score : 0.95684
accuracy score : 0.92796
thre: 0.9969162514
fp:  0.0009850342
recall:  0.1083908575


(0.9568397385651055, 0.2584200520476898, 0.1083908574934806)

## Malcon

In [13]:
import keras
import numpy as np

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'

    def __init__(self, list_IDs, datasets, labels, batch_size=32, dim=8192, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.datasets = datasets
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'  # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.zeros((self.batch_size, self.dim), dtype=float)
        y = np.zeros(self.batch_size, dtype=float)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            base_path = "/ssd/2017/{0}/{1}{2}"
            item = self.datasets.loc[ID]
            file_path = base_path.format(item["mw_file_directory"], item["mw_file_hash"], item["mw_file_size"])
            in_file = open(file_path, 'rb')
            in_file.seek(item['pointerto_raw_data'])
            if item['virtual_size'] > max_length:
                bytes_data = [int(single_byte) for single_byte in in_file.read(max_length)]
            else:
                bytes_data = [int(single_byte) for single_byte in in_file.read(item['virtual_size'])]
            X[i, 0:len(bytes_data)] = bytes_data
            y[i] = self.labels[ID]

        return X, y

Using TensorFlow backend.


In [14]:
import hashlib
import json
import time

import keras
from keras import Input
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.layers import Dense, Embedding, Conv1D, Multiply, GlobalMaxPooling1D, Dropout
from keras.models import load_model
from sklearn.model_selection import train_test_split

class TMalConv(object):
    """
    train of mal conv
    """

    def __init__(self):
        self.max_len = max_length
        self.history = None
        self.model = None
        self.p_md5 = None
        self.time = time.time()
        self.summary = {
            'time':time.time(),
            'batch_size': 32,
            'epochs': 64,
            'g_c_filter': 128,
            'g_c_kernel_size': 500,
            'g_c_stride': 500,
        }

    def run(self):
        """
        :return:
        """
        self.train()
        
    def get_p(self, key):
        """
        get the parameter from the summary
        :param key:
        :return:
        """
        return self.summary[key]

    def gate_cnn(self, gate_cnn_input):
        """
        construct a gated cnn by the specific kernel size
        :param gate_cnn_input:
        :param kernel_size:
        :return:
        """
        conv1_out = Conv1D(self.get_p("g_c_filter"), self.get_p("g_c_kernel_size"), strides=self.get_p("g_c_stride"))(
            gate_cnn_input)
        conv2_out = Conv1D(self.get_p("g_c_filter"), self.get_p("g_c_kernel_size"), strides=self.get_p("g_c_stride"),
                           activation="sigmoid")(gate_cnn_input)
        merged = Multiply()([conv1_out, conv2_out])
        gate_cnn_output = GlobalMaxPooling1D()(merged)
        return gate_cnn_output

    def get_model(self):
        """
        get a model
        :param max_len:
        :param kernel_sizes:
        :return:
        """
        net_input = Input(shape=(self.max_len,))

        embedding_out = Embedding(256, 8, input_length=self.max_len)(net_input)
        merged = self.gate_cnn(embedding_out)

        dense_out = Dense(128)(merged)
        
        net_output = Dense(1, activation='sigmoid')(dense_out)

        model = keras.models.Model(inputs=net_input, outputs=net_output)

        return model

    def train(self):
        batch_size = self.get_p("batch_size")
        epochs = self.get_p("epochs")

        self.model = self.get_model()

        print('Length of the train: ', len(x_train))
        print('Length of the validation: ', len(x_val))
        
#         tensor_board = TensorBoard(log_dir='./logs/', batch_size=batch_size)
        file_path = "/home/zhaoqi/BaseTrain/models/"+ str(self.time) +"-{epoch:04d}-{val_loss:.5f}-{val_acc:.5f}.h5"
        early_stopping = EarlyStopping("val_loss", patience=3, verbose=0, mode='auto')
        check_point = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
        callbacks_list = [check_point, early_stopping]

        # Generators
        training_generator = DataGenerator(range(len(x_train)), x_train, y_train, batch_size, self.max_len)
        validation_generator = DataGenerator(range(len(x_val)), x_val, y_val, batch_size, self.max_len)

        self.model.compile(loss='binary_crossentropy',
                           optimizer='adam',
                           metrics=['accuracy'])

        self.model.fit_generator(generator=training_generator,
                                 validation_data=validation_generator,
                                 use_multiprocessing=True,
                                 epochs=epochs,
                                 workers=6,
                                 callbacks=callbacks_list)

In [15]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

In [18]:
t_instance = TMalConv()
t_instance.run()

Length of the train:  241580
Length of the validation:  26843
Epoch 1/64
7549/7549 [==============================] - 7011s 929ms/step - loss: 0.1698 - acc: 0.9464 - val_loss: 0.1518 - val_acc: 0.9534

Epoch 00001: val_loss improved from inf to 0.15181, saving model to /home/zhaoqi/BaseTrain/models/1532328550.5178833-0001-0.15181-0.95342.h5
Epoch 2/64
7549/7549 [==============================] - 6100s 808ms/step - loss: 0.1228 - acc: 0.9622 - val_loss: 0.1467 - val_acc: 0.9555

Epoch 00002: val_loss improved from 0.15181 to 0.14671, saving model to /home/zhaoqi/BaseTrain/models/1532328550.5178833-0002-0.14671-0.95551.h5
Epoch 3/64
7549/7549 [==============================] - 6040s 800ms/step - loss: 0.0957 - acc: 0.9729 - val_loss: 0.1433 - val_acc: 0.9568

Epoch 00003: val_loss improved from 0.14671 to 0.14332, saving model to /home/zhaoqi/BaseTrain/models/1532328550.5178833-0003-0.14332-0.95678.h5
Epoch 4/64
 287/7549 [>.............................] - ETA: 1:16:28 - loss: 0.0798 - a

Process ForkPoolWorker-38:
Process ForkPoolWorker-31:
Process ForkPoolWorker-33:
Process ForkPoolWorker-32:
Process ForkPoolWorker-36:
Process ForkPoolWorker-39:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/multiprocessing/pool.py", line 119, in worker

KeyboardInterrupt: 

In [16]:
model_dir = '/home/zhaoqi/BaseTrain/models/'
f_name = '1532328550.5178833-0003-0.14332-0.95678.h5'
c_model = load_model(model_dir + f_name)

# test_generator = DataGenerator(range(len(x_test)), x_test, y_test, 32, max_length, False)
# y_pred = c_model.predict_generator(generator=test_generator, max_queue_size=10, workers=6, use_multiprocessing=True, verbose=1)

In [20]:
estimate_model(y_pred, y_test[0:len(y_pred)])

loss : 0.33252
auc score : 0.94611
accuracy score : 0.89069
thre: 0.9984807372
fp:  0.0002911013
recall:  0.0323258668


(0.9461135205276512, 0.3325151933146175, 0.03232586683031605)

## Merge features

### Merge Malconv and Ember

In [17]:
from keras.models import Model

model_f = Model(c_model.input, c_model.layers[-2].output)

train_generator = DataGenerator(range(len(x_train)), x_train, y_train, 32, max_length, False)
malcon_train_x = model_f.predict_generator(generator=train_generator, max_queue_size=10, workers=6, use_multiprocessing=True, verbose=1)

val_generator = DataGenerator(range(len(x_val)), x_val, y_val, 32, max_length, False)
malcon_val_x = model_f.predict_generator(generator=val_generator, max_queue_size=10, workers=6, use_multiprocessing=True, verbose=1)

test_generator = DataGenerator(range(len(x_test)), x_test, y_test, 32, max_length, False)
malcon_test_x = model_f.predict_generator(generator=test_generator, max_queue_size=10, workers=6, use_multiprocessing=True, verbose=1)

4828/4828 [==============================] - 2550s 528ms/step


In [18]:
def merge_feature(m_data, e_data):
    num = len(m_data)
    m_x = np.zeros((num, 128+2351), dtype=float)
    
    for index in range(num):
        m_x[index, 0:128] = m_data[index]
        m_x[index, 128:128+2351] = e_data[index]  
    return m_x

In [19]:
merge_train_x = merge_feature(malcon_train_x, x_etrain)
merge_val_x = merge_feature(malcon_val_x, x_eval)
merge_test_x = merge_feature(malcon_test_x, x_etest)

model_m = get_model(merge_train_x, y_train[0:len(merge_train_x)], merge_val_x, y_val[0:len(merge_val_x)] )

[1]	valid_0's binary_logloss: 0.573842
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's binary_logloss: 0.505857
[3]	valid_0's binary_logloss: 0.450531
[4]	valid_0's binary_logloss: 0.404264
[5]	valid_0's binary_logloss: 0.36538
[6]	valid_0's binary_logloss: 0.332398
[7]	valid_0's binary_logloss: 0.304012
[8]	valid_0's binary_logloss: 0.279572
[9]	valid_0's binary_logloss: 0.258307
[10]	valid_0's binary_logloss: 0.239977
[11]	valid_0's binary_logloss: 0.223902
[12]	valid_0's binary_logloss: 0.209692
[13]	valid_0's binary_logloss: 0.197407
[14]	valid_0's binary_logloss: 0.186504
[15]	valid_0's binary_logloss: 0.177078
[16]	valid_0's binary_logloss: 0.168876
[17]	valid_0's binary_logloss: 0.161752
[18]	valid_0's binary_logloss: 0.155464
[19]	valid_0's binary_logloss: 0.149885
[20]	valid_0's binary_logloss: 0.144831
[21]	valid_0's binary_logloss: 0.140344
[22]	valid_0's binary_logloss: 0.136735
[23]	valid_0's binary_logloss: 0.133351
[24]	valid_0's binary_loglos

In [20]:
y_p = model_m.predict(merge_test_x)
y_pred = np.zeros((len(y_p), 1))
for i in range(len(y_p)):
    y_pred[i, 0] = y_p[i]

estimate_model(y_pred, y_test[0:len(merge_test_x)])

loss : 0.34086
auc score : 0.95414
accuracy score : 0.90378
thre: 0.9939017798
fp:  0.0008509114
recall:  0.1281067812


(0.9541435176854167, 0.340863601189653, 0.1281067812212335)